In [49]:
import os,shutil,pickle
import pandas as pd
import datetime
from datetime import timedelta
import names

## Load Data

In [52]:
path = '../static/model'

re_1 = pd.read_csv(os.path.join(path,'submission50_mew_v2.csv'),dtype=str).drop(columns=['Unnamed: 0'])
cus_list = re_1['customer_id'].unique()



re_2 = pd.read_csv(os.path.join(path,'submission50_eye.csv'),dtype=str).drop(columns=['Unnamed: 0','recent_purchase'])
# re_2['recent_purchase'] = re_2['recent_purchase'].apply(lambda x: '0'+x)
re_2['prediction'] = re_2['prediction'].apply(lambda x: ' '.join(['0'+i for i in x.split()]))
re_2 = re_2.merge(re_1[['customer_id','recent_purchase']],on='customer_id',how='left')

re_3 = pd.read_csv(os.path.join(path,'submission50_got.csv'),dtype=str).drop(columns=['Unnamed: 0','recent_purchase'])
# re_3['recent_purchase'] = re_3['recent_purchase'].apply(lambda x: ' '.join(['0'+i for i in x.split()]))
re_3['prediction'] = re_3['prediction'].apply(lambda x: ' '.join(['0'+i for i in x.split()]))
re_3 = re_3.merge(re_1[['customer_id','recent_purchase']],on='customer_id',how='left')



re_4 = pd.read_csv(os.path.join(path,'submission50_non.csv'),dtype=str)

re_5 = pd.read_csv(os.path.join(path,'submission50_bird.csv'),dtype=str).rename(columns={'last_article':'recent_purchase','recomends':'prediction'})
re_5 = re_5[['customer_id','prediction','recent_purchase']][re_5.customer_id.isin(cus_list)]
re_5['recent_purchase'] = re_5['recent_purchase'].apply(lambda x: '0'+x)
re_5['prediction'] = re_5['prediction'].apply(lambda x: ' '.join(['0'+i for i in x.split()]))

re_3.head()

,customer_id,prediction,recent_purchase
0,01343722529be320bc0955acc1b7c40322e5f801c6b8e0...,0706016001 0692930001 0448509014 0689109001 08...,0765739001 0708246001 0801384004 0738731002 07...
1,0a31e048711796ab3ec0179034fd9d7282101e42c89684...,0810419002 0529012011 0717695002 0811907006 08...,0516859008 0529008003 0703366002 0682238007 05...
2,0c86d1502d0f9c23157fb89bd97e45587372777b318a83...,0833499006 0757482002 0737224003 0547367021 05...,0637515004 0716548003 0547367006 0723469007 05...
3,0ebd7f3073819f86ba5f2ad025db2b51ae36ad7039d453...,0710278004 0750330002 0573937001 0751551001 06...,0842314001 0873884005 0746330004 0929042001 08...
4,10f245b02120d5c89b574db89b290983238a0a4e98a935...,0599580055 0599580042 0559630026 0776237020 05...,0761624001 0907696002 0776237010 0802485004 07...


## 7-Days Target

In [53]:
# mapping index
path = '../data/processed'
infile = open(os.path.join(path,'index_to_cusId.pkl'),'rb')
index_to_id_dict = pickle.load(infile)
infile.close()

In [54]:
path = '../data/processed'
trans = pd.read_pickle(os.path.join(path,'transactions.pkl'))
trans["customer_id"] = trans["customer_id"].map(index_to_id_dict)
trans.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [55]:
start_dt =  datetime.datetime(2020,9,15)
end_dt = start_dt + timedelta(7)

trans = trans[(trans.t_dat > start_dt) & (trans.t_dat <= end_dt)]
print('Min date: ', trans.t_dat.min())
print('Max date: ', trans.t_dat.max())

target = pd.DataFrame(trans.groupby(['customer_id'])['article_id'].apply(lambda x: list(set(x))))\
                        .reset_index()\
                        .rename(columns={'article_id':'actual'})
# weekly_purchased['weekly_purchased_products'] = weekly_purchased['weekly_purchased_products'].apply(lambda x: list(set(x)))
target = target[target.customer_id.isin(cus_list)]
print(f'Total Customers: {target.customer_id.nunique()}')
target.head()

Min date:  2020-09-16 00:00:00
Max date:  2020-09-22 00:00:00
Total Customers: 50


,customer_id,actual
338,01343722529be320bc0955acc1b7c40322e5f801c6b8e0...,"[0928210002, 0874754002, 0714790020, 092238100..."
2758,0a31e048711796ab3ec0179034fd9d7282101e42c89684...,"[0730570029, 0759814020, 0794575002, 078290100..."
3364,0c86d1502d0f9c23157fb89bd97e45587372777b318a83...,"[0877268005, 0685813001, 0685813042, 092313400..."
3970,0ebd7f3073819f86ba5f2ad025db2b51ae36ad7039d453...,"[0790368006, 0894320001, 0919273004, 087388400..."
4567,10f245b02120d5c89b574db89b290983238a0a4e98a935...,"[0852174001, 0757303021, 0863284001, 085258400..."


## Random name

In [56]:
# random name

cus_name = {}
for i in cus_list:
    cus_name[i] = names.get_full_name()
cus_name = pd.DataFrame.from_dict(cus_name,orient='index',columns=['name']).reset_index(drop=False).rename(columns={'index':'customer_id'})

cus_name.head()

,customer_id,name
0,01343722529be320bc0955acc1b7c40322e5f801c6b8e0...,Lisa Holderby
1,0a31e048711796ab3ec0179034fd9d7282101e42c89684...,Wilfred Parks
2,0c86d1502d0f9c23157fb89bd97e45587372777b318a83...,Leonard Murphy
3,0ebd7f3073819f86ba5f2ad025db2b51ae36ad7039d453...,Janet Boggioni
4,10f245b02120d5c89b574db89b290983238a0a4e98a935...,Diane Alvarado


## Consolidate data

In [57]:
target['actual'] = target['actual'].apply(lambda x: ' '.join(x))

In [58]:
re_1['model'] = 'ALS Model'
re_2['model'] = 'User-User Similarity Model'
re_3['model'] = 'Ranking Factorization Model'
re_4['model'] = 'Image Similarity Model'
re_5['model'] = 'Item-Item Similarity Model'





frames = [re_1, re_2, re_3,re_4,re_5]
for i in frames:
    print(len(i))
result = pd.concat(frames)


result = result.merge(target,on='customer_id',how='left')
result = result.merge(cus_name,on='customer_id',how='left')
# result['actual'] = result['actual'].fillna('')
result

50
50
49
50
50


,customer_id,prediction,recent_purchase,model,actual,name
0,01343722529be320bc0955acc1b7c40322e5f801c6b8e0...,0894668003 0714790028 0874754002 0714790020 07...,0765739001 0708246001 0801384004 0738731002 07...,ALS Model,0928210002 0874754002 0714790020 0922381001 08...,Lisa Holderby
1,0a31e048711796ab3ec0179034fd9d7282101e42c89684...,0794575002 0682550002 0621381016 0621381012 07...,0516859008 0529008003 0703366002 0682238007 05...,ALS Model,0730570029 0759814020 0794575002 0782901006 08...,Wilfred Parks
2,0c86d1502d0f9c23157fb89bd97e45587372777b318a83...,0685813001 0685813042 0685813003 0685813040 06...,0637515004 0716548003 0547367006 0723469007 05...,ALS Model,0877268005 0685813001 0685813042 0923134007 06...,Leonard Murphy
3,0ebd7f3073819f86ba5f2ad025db2b51ae36ad7039d453...,0919273002 0901330002 0790368006 0919273004 09...,0842314001 0873884005 0746330004 0929042001 08...,ALS Model,0790368006 0894320001 0919273004 0873884005 09...,Janet Boggioni
4,10f245b02120d5c89b574db89b290983238a0a4e98a935...,0740519002 0730683052 0852174001 0804992017 08...,0761624001 0907696002 0776237010 0802485004 07...,ALS Model,0852174001 0757303021 0863284001 0852584002 07...,Diane Alvarado
...,...,...,...,...,...,...
244,ec2f388a3b3e632bbe818173306713898cf2e12fd58c4a...,0516789002 0612064008 0618387003 0633535003 06...,0816166002,Item-Item Similarity Model,0855080011 0855080009 0855080002 0921697003 08...,William Allen
245,f160f1a3bb3a7e2a7558c3ae98c3531dd920a43654732e...,0740564002 0740943002 0772794003 0857812007 08...,0740943002,Item-Item Similarity Model,0905889002 0870328002 0870328001 0884319003 08...,Walter Avila
246,fa06f8154a9fb9de61d24362e924a7c1dfaa51e6bc6189...,0544054013 0560030021 0621735015 0634013005 06...,0795836005,Item-Item Similarity Model,0562245104 0883079006 0929165002 0794575002 09...,Henry Tacadina
247,fced61dd06e3e0d5628802f239afb1c023b7055719e8ab...,0188183001 0205225001 0351484002 0377289002 04...,0600886001,Item-Item Similarity Model,0854193004 0812668002 0852374004 0837278001 08...,Charles Peterson


## Get Image Files

In [59]:
unique_articles = []
for i in result.prediction:
    unique_articles = unique_articles + i.split()
    
for i in result.recent_purchase:
    if len(i) > 10:
        temp = i.split()
    else:
        temp = [i]
    unique_articles = unique_articles + temp 
    
for i in target.actual:
    if len(i) > 10:
        temp = i.split()
    else:
        temp = [i]
    unique_articles = unique_articles + temp

unique_articles = list(set(unique_articles))
print('Total unique_articles: ', len(unique_articles))
# unique_articles

Total unique_articles:  2531


In [60]:
from tqdm import tqdm
from PIL import Image

save_path = '../static/model/images'
raw_path = '../data/raw/images'

width = 1166
hieght = 1750
reduce = 0.2

# Clear all file in the directory
for f in os.listdir(save_path):
    os.remove(os.path.join(save_path, f))

# Copy using picture
for i in unique_articles:
    src = os.path.join(raw_path,i[:3],i+'.jpg')
    dst = os.path.join(save_path,i+'.jpg')
    if os.path.exists(src):
        shutil.copyfile(src, dst)
    else:
        print(f'Missing articles: {i}')

# Resize Image
multiple_images = os.listdir(save_path)

# Looping over all of the images:
for image in multiple_images:
    img = Image.open(os.path.join(save_path, image))
    img.thumbnail(size=(width*reduce,hieght*reduce))
#     print(img)
    # We would run the command below to save the images:
    img.save(os.path.join(save_path, image), optimize=True)
print('Done. Resize Image')

Missing articles: 0513701002
Missing articles: 0626263002
Missing articles: 0863646002
Missing articles: 0610776002
Missing articles: 0179208001
Missing articles: 0446224011
Missing articles: 0218354021
Missing articles: 0610776001
Missing articles: 0216961011
Done. Resize Image


## Get articles

In [61]:
# articles description
articles = pd.read_pickle(os.path.join(path,'articles.pkl'))
articles.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [62]:
select_articles = articles[articles.article_id.isin(unique_articles)].reset_index(drop=True)
print('Total Selected Articles: ',len(select_articles))
select_articles.head()

Total Selected Articles:  2531


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
3,0111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
4,0111593001,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


## Save file

In [63]:
final_save_path = '../static/model'

result.to_csv(os.path.join(final_save_path,'prediction.csv'))
select_articles.to_csv(os.path.join(final_save_path,'articles_v2.csv'))
# recent_purchase.to_csv(os.path.join(final_save_path,'recent_purchase.csv'))
cus_name.to_csv(os.path.join(final_save_path,'customers_v2.csv'))

In [64]:
mylist=['ALS Model','Ranking Factorization','User-User Similarity','Item-Item Similarity','Image Similarity']
targetvalue= 'Image Similarity'
mylist.insert(0, mylist.pop(mylist.index(targetvalue)))
mylist

['Image Similarity',
 'ALS Model',
 'Ranking Factorization',
 'User-User Similarity',
 'Item-Item Similarity']

In [65]:
mylist=['ALS Model','Ranking Factorization','User-User Similarity','Item-Item Similarity','Image Similarity']
for i in mylist:
    print(i + ' Model')

ALS Model Model
Ranking Factorization Model
User-User Similarity Model
Item-Item Similarity Model
Image Similarity Model
